In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.callbacks import EarlyStopping
from keras_tuner import RandomSearch
import random
from keras.optimizers import Adam

# Define the file paths for each period
file_paths = {
    'bear_market_1': '../../../Data/Final_df/bear_market_1_regression.csv',
    'bear_market_2': '../../../Data/Final_df/bear_market_2_regression.csv',
    'bull_market_1': '../../../Data/Final_df/bull_market_1_regression.csv',
    'bull_market_2': '../../../Data/Final_df/bull_market_2_regression.csv'
}

# Initialize the results dictionary
results = {}

# Seed value for reproducibility
seed_value = 42
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

# Normalize the data
columns_to_scale = ['Open', 'High', 'Low', 'Close','Volume', 'RSI', 'ATR', 'MACD', 'MFI',
                    'EMA', 'SMA', 'OBV', 'GTrends_Interest', 'Sentiment_Bullish',
                    'Price_oil', 'Price_gold', 'Price_NASDAQ', 'Price_SP500', 'Price_NYSE',
                    'Interest_Rate', 'hash_rate', 'users']

scaler = MinMaxScaler(feature_range=(0, 1))

# Define the function to create the dataset
def create_dataset(data, window_size, target_index):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data.iloc[i:(i + window_size)].values)
        y.append(data.iloc[i + window_size, target_index])
    return np.array(X), np.array(y)

# Forecast Horizon
window_size = 3
close_index = columns_to_scale.index('Close')

# Define a model-building function
def build_model(hp):
    model = Sequential()
    units = hp.Int('units', min_value=10, max_value=200, step=10)
    
    model.add(LSTM(units=units, return_sequences=True, input_shape=(window_size, len(columns_to_scale))))
    model.add(LSTM(units=units, return_sequences=True))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dense(1, activation='linear'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                  loss='mean_squared_error', metrics=['mae', 'mean_absolute_percentage_error'])
    return model

# Function to manually inverse transform the values
def manual_inverse_transform(scaled_values, min_val, max_val):
    return scaled_values * (max_val - min_val) + min_val

# Function to fit the model and evaluate metrics
def fit_and_evaluate(data, scaler, window_size, close_index, project_name):
    data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])
    
    # Extract min and max values for 'Close'
    close_min = scaler.data_min_[close_index]
    close_max = scaler.data_max_[close_index]
    
    print(f"Period: {project_name}")
    print(f"Close Min: {close_min}, Close Max: {close_max}")

    training_size = int(len(data) * 0.9)
    training_data = data[:training_size]
    test_data = data[training_size:]
    train_data = training_data[:int(len(training_data) * 0.9)]
    val_data = training_data[int(len(training_data) * 0.9):]

    X_train, y_train = create_dataset(train_data, window_size, close_index)
    X_test, y_test = create_dataset(test_data, window_size, close_index)
    X_val, y_val = create_dataset(val_data, window_size, close_index)
    X_train_full, y_train_full = create_dataset(training_data, window_size, close_index)

    X_train = X_train.reshape((X_train.shape[0], window_size, X_train.shape[2]))
    X_test = X_test.reshape((X_test.shape[0], window_size, X_test.shape[2]))
    X_val = X_val.reshape((X_val.shape[0], window_size, X_val.shape[2]))
    X_train_full = X_train_full.reshape((X_train_full.shape[0], window_size, X_train_full.shape[2]))

    tuner = RandomSearch(
        build_model,
        objective='val_mean_absolute_percentage_error',
        max_trials=200,
        executions_per_trial=1,
        directory='my_dir',
        project_name=project_name,
        overwrite=False
    )

    early_stopping = EarlyStopping(monitor='val_mean_absolute_percentage_error', patience=15, restore_best_weights=True)

    tuner.search(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_val, y_val), verbose=2, callbacks=[early_stopping])

    best_hps = tuner.get_best_hyperparameters()[0]

    mae_list, rmse_list, mape_list = [], [], []
    for _ in range(10):
        best_model = tuner.hypermodel.build(best_hps) 
        history = best_model.fit(X_train_full, y_train_full, epochs=250, batch_size=32, verbose=0)
        predictions = best_model.predict(X_test)

        # Manually inverse transform the predictions and y_test
        predictions_unscaled = manual_inverse_transform(predictions, close_min, close_max)
        y_test_unscaled = manual_inverse_transform(y_test.reshape(-1, 1), close_min, close_max)

        # Calculate unscaled metrics
        mae_unscaled = mean_absolute_error(y_test_unscaled, predictions_unscaled)
        rmse_unscaled = np.sqrt(mean_squared_error(y_test_unscaled, predictions_unscaled))
        mape_unscaled = tf.keras.losses.MAPE(y_test_unscaled, predictions_unscaled).numpy()

        mae_list.append(mae_unscaled)
        rmse_list.append(rmse_unscaled)
        mape_list.append(mape_unscaled)

    return {
        'mae': np.mean(mae_list),
        'rmse': np.mean(rmse_list),
        'mape': np.mean(mape_list)
    }

# Process each period and store results
for period, file_path in file_paths.items():
    data = pd.read_csv(file_path)
    data['Date'] = pd.to_datetime(data['Date'])
    data.set_index('Date', inplace=True)
    project_name = f'LSTM-3D-REG-{period}'
    results[period] = fit_and_evaluate(data, scaler, window_size, close_index, project_name)

# Create a DataFrame from the results
results_df = pd.DataFrame(results).T
print(results_df)

# Save the results to a CSV file
#results_df.to_csv('model_performance_summary_lstm_regression.csv')


Period: LSTM-3D-REG-bear_market_1
Close Min: 3236.761719, Close Max: 19140.800781
Reloading Tuner from my_dir/LSTM-3D-REG-bear_market_1/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step


...

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
                       mae         rmse       mape
bear_market_1  1262.435952  1430.340303  33.344440
bear_market_2  1220.801500  1448.053805   6.394872
bull_market_1  1112.959303  1716.552592  12.106766
bull_market_2  5194.987615  5885.535595  10.757636


In [2]:
# Save the besto model for each period
# Bear Market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='LSTM-3D-REG-bear_market_1',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_LSTM-3D-REG-bear_market_1.keras')

# Bear Market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='LSTM-3D-REG-bear_market_2',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_LSTM-3D-REG-bear_market_2.keras')

# Bull Market 1
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='LSTM-3D-REG-bull_market_1',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_LSTM-3D-REG-bull_market_1.keras')

# Bull Market 2
tuner = RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=200,
    executions_per_trial=1,
    directory='my_dir',
    project_name='LSTM-3D-REG-bull_market_2',
    overwrite=False
)
best_hps = tuner.get_best_hyperparameters()[0]
best_model = tuner.hypermodel.build(best_hps)
best_model.save('best_LSTM-3D-REG-bull_market_2.keras')

Reloading Tuner from my_dir/LSTM-3D-REG-bear_market_1/tuner0.json
Reloading Tuner from my_dir/LSTM-3D-REG-bear_market_2/tuner0.json
Reloading Tuner from my_dir/LSTM-3D-REG-bull_market_1/tuner0.json


/opt/anaconda3/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Reloading Tuner from my_dir/LSTM-3D-REG-bull_market_2/tuner0.json
